# Blahut-Arimato algorithm for channel capacity 

This notebook calculates the channel capacity of channel with a given transition matrix, both with a forward and a backward variant

First version: 11/08/2025

This version: 11/17/2025

https://northeastern-datalab.github.io/cs7840/fa25/calendar.html

## 1. Multinomial distribution on the 3-simplex

In [1]:
import numpy as np

# ---------- helper functions ----------
def safe_log2(x):
    """
    Elementwise log2 of a vector x for entropy-like sums
    with imposed convention 0*log2(0)=0 
    """
    out = np.zeros_like(x, dtype=float)
    mask = x > 0.0
    out[mask] = np.log2(x[mask])
    return out


def D_rows_bits(P, q):
    """
    D_x = D_KL(P(row x) || q) in bits, i.e., D_x = sum_y P[x,y] * log2(P[x,y]/q[y]).
    """
    m, n = P.shape
    D = np.zeros(m, dtype=float)
    logq = safe_log2(q)
    for x in range(m):
        Px = P[x]           # stochastic row vector
        mask = Px > 0
        D[x] = np.sum(Px[mask] * (safe_log2(Px[mask]) - logq[mask]))
    return D

def capacity_bits(p, P):
    """
    I(X;Y) = H(Y) - H(Y|X) in bits for current p and channel P.
    """
    q = p @ P                         # output distribution
    HY = -np.dot(q, safe_log2(q))     # H(Y)
    # row entropies H(P(row x)) then average under p
    row_H = -np.sum(P * safe_log2(P), axis=1)
    HYgX = float(np.dot(p, row_H))
    return HY - HYgX


# ---------- Forward BA: multiplicative-weights (Arimoto/Dauwels) ----------
def ba_forward(P, tol=1e-12, max_iters=10000, verbose=True):
    """
    Forward BA (multiplicative-weights) in bits.
    Starts from uniform p, prints p and C each iteration, returns (p*, C*).
    """
    P = np.asarray(P, dtype=float)
    m, n = P.shape
    p = np.ones(m, dtype=float) / m  # uniform start

    for t in range(max_iters):
        q = p @ P
        D = D_rows_bits(P, q)
        C = float(np.dot(p, D))
        if verbose:
            print(f"iter {t:4d}: p = {np.round(p, 12)},  C = {C:.12f} bits")
        # multiplicative-weights update (base-2)
        w = p * (2.0 ** D)
        p_new = w / w.sum()
        
        if np.linalg.norm(p_new - p, 1) < tol:
            p = p_new
            # final print at convergence
            q = p @ P
            D = D_rows_bits(P, q)
            C = float(np.dot(p, D))
            if verbose:
                print(f"iter {t+1:4d}: p = {np.round(p, 12)},  C = {C:.12f} bits  (converged)")
            return p, C
        p = p_new

    # max_iters reached
    q = p @ P
    D = D_rows_bits(P, q)
    C = float(np.dot(p, D))
    if verbose:
        print(f"[stopped: max_iters] p = {np.round(p, 12)},  C = {C:.12f} bits")
    return p, C


# ---------- Reverse BA: posterior form (Wikipedia/EM-style) ----------
def ba_reverse(P, tol=1e-12, max_iters=10000, verbose=True):
    """
    Reverse BA (posterior/inverse form) in bits.
    Starts from uniform p, prints p and C each iteration, returns (p*, C*).
    """
    P = np.asarray(P, dtype=float)
    m, n = P.shape
    p = np.ones(m, dtype=float) / m  # uniform start

    for t in range(max_iters):
        # E-step: q and Q(x|y) via Bayes using the forward channel P
        q = p @ P
        # Avoid division-by-zero if a q_y is zero (rare for proper channels)
        denom = q.copy()
        eps = 0.0
        denom[denom == 0.0] = eps
        Q = (p[:, None] * P) / denom[None, :]   # shape m x n, columns sum to 1

        # Capacity at current p (for printing)
        D = D_rows_bits(P, q)
        C = float(np.dot(p, D))
        if verbose:
            print(f"iter {t:4d}: p = {np.round(p, 12)},  C = {C:.12f} bits")

        # M-step: p^{new}(x) ∝ 2^{ S_x },  S_x = sum_y P(x,y) log2 Q(x|y)
        S = np.sum(P * safe_log2(Q), axis=1)
        w = 2.0 ** S
        p_new = w / w.sum()

        if np.linalg.norm(p_new - p, 1) < tol:
            p = p_new
            # final print at convergence
            q = p @ P
            D = D_rows_bits(P, q)
            C = float(np.dot(p, D))
            if verbose:
                print(f"iter {t+1:4d}: p = {np.round(p, 12)},  C = {C:.12f} bits  (converged)")
            return p, C
        p = p_new

    # max_iters reached
    q = p @ P
    D = D_rows_bits(P, q)
    C = float(np.dot(p, D))
    if verbose:
        print(f"[stopped: max_iters] p = {np.round(p, 12)},  C = {C:.12f} bits")
    return p, C


## Example test cases

In [2]:
CASE = 1

if CASE == 1:
    P = np.array([
        [0.7, 0.2, 0.1],
        [0.3, 0.6, 0.1],
        [0.3, 0.2, 0.5]
    ], dtype=float)

elif CASE == 2:
    P = np.array([
        [0.6, 0.3, 0.1],
        [0.1, 0.6, 0.3],
        [0.3, 0.1, 0.6]
    ], dtype=float)


print("=== Forward BA ===")
p_f, C_f = ba_forward(P, tol=1e-4)

print("\n=== Reverse BA ===")
p_r, C_r = ba_reverse(P, tol=1e-4)


=== Forward BA ===
iter    0: p = [0.33333333 0.33333333 0.33333333],  C = 0.228435607334 bits
iter    1: p = [0.33015987 0.33308427 0.33675586],  C = 0.228521622459 bits
iter    2: p = [0.32750076 0.33297704 0.3395222 ],  C = 0.228579693974 bits
iter    3: p = [0.32526862 0.33297411 0.34175726],  C = 0.228619097824 bits
iter    4: p = [0.32339152 0.33304554 0.34356294],  C = 0.228645984885 bits
iter    5: p = [0.32181019 0.33316782 0.34502199],  C = 0.228664441350 bits
iter    6: p = [0.32047574 0.33332284 0.34620142],  C = 0.228677189882 bits
iter    7: p = [0.31934779 0.33349685 0.34715536],  C = 0.228686051722 bits
iter    8: p = [0.31839289 0.33367963 0.34792748],  C = 0.228692250848 bits
iter    9: p = [0.31758331 0.33386372 0.34855297],  C = 0.228696614224 bits
iter   10: p = [0.31689599 0.33404385 0.34906016],  C = 0.228699703824 bits
iter   11: p = [0.31631171 0.33421642 0.34947187],  C = 0.228701903906 bits
iter   12: p = [0.31581443 0.33437912 0.34980645],  C = 0.22870347889